# Computing NPV of Swap Trades

In this session we will look at how to calculate Net Present Value (NPV) of swap trades using OpenRedukti.

### Overview of steps

The process involves following steps:

* First we will construct zero curves using the steps described in the previous session.
* Then we will register various data structures with the Redukti GRPC server
* Finally we will construct some swap trades and compute their NPVs

### Checking that server is up

In [1]:
from redukti.library import ServerCommand

In [2]:
server_command = ServerCommand('localhost:9001')

In [3]:
server_command.hello('hello redukti')

'hello redukti'

The server has echoed back our input so that means it is running.

### Loading Market Data 

We can now load the curve definitions and par rates into memory structures using a library function provided by redukti.

In [4]:
from redukti.library import MarketData

In [5]:
import redukti

In [6]:
market_data = MarketData(redukti.dmy(11,12,2012)) # business date is 11th Dec 2012

In [7]:
market_data.read_curvedefinitions('../testdata/20121211/curve_definitions.csv')

In [8]:
market_data.read_parcurves('../testdata/20121211/par_rates.csv')

We can also load fixings data. These are available in a file.

In [13]:
market_data.read_fixings('../testdata/20121211/fixings.csv')

### Building Curves

We can now request the server to bootstrap the curves. 

In [9]:
server_command.build_curves(market_data)

Curve 7 created
Curve 8 created
Curve 9 created
Curve 10 created


### Register Market Data With Redukti Server

Now that our curves are ready we can register those with the Redukti Server:

In [10]:
from redukti import enums_pb2 as enums

We need to make a list of the curves that will be used for estimating forward rates. In our case these are curves with ids 8,9, and 10

In [11]:
forward_curves = [8,9,10]

We also need a list of curves that will be used for discounting cashflows. In our case this is the EONIA curve which has id 7.

In [12]:
discount_curves = [7]

We have all the market data we need for computing NPVs, but we must load these into the Redukti Server. The Redukti Server provides apis for loading curve definitions, zero rates and fixings. These are wrapped in a method named `register_market_data()` which we will invoke now.

In [14]:
server_command.register_market_data(enums.CURVE_GROUP_C, market_data, forward_curves, discount_curves)

Registering curve definitions
Registering zero curves
Registering fixings
